# 1. import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# genral packages
import ast
import csv
import h5py
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
# pytorch, __version__ = '1.8.1+cu101'
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
# Use GPU
using_GPU = torch.cuda.is_available()
using_GPU

True

In [ ]:
# __version__ = '2.4.0'
!pip install allennlp==2.4.0

     |████████████████████████████████| 634kB 8.1MB/s 
     |████████████████████████████████| 122kB 15.6MB/s 
     |████████████████████████████████| 1.8MB 17.7MB/s 
     |████████████████████████████████| 266kB 33.1MB/s 
     |████████████████████████████████| 133kB 35.2MB/s 
     |████████████████████████████████| 2.1MB 41.1MB/s 
     |████████████████████████████████| 1.2MB 35.9MB/s 
     |████████████████████████████████| 133kB 37.1MB/s 
     |████████████████████████████████| 174kB 40.3MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 7.6MB 32.4MB/s 
     |████████████████████████████████| 901kB 31.2MB/s 
     |████████████████████████████████| 3.3MB 32.1MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp37-cp37m-linux_x86_64.whl size=3388786 sha256=b79a6409936963a4a06b108ce37789a5769c5e3ab3f0190b21edc530aac5de5

In [ ]:
from allennlp.nn.util import sort_batch_by_length

# 2. Model

## attention setting

In [ ]:
class Attention(nn.Module):

  # Modified based on TorchNLP https://github.com/kolloldas/torchnlp

  def __init__(self, input_depth, total_key_depth, total_value_depth, output_depth, 
         num_heads, bias_mask=None, dropout=0.0):
    """
    Parameters:
      input_depth: Size of last dimension of input
      total_key_depth: Size of last dimension of keys. Must be divisible by num_head
      total_value_depth: Size of last dimension of values. Must be divisible by num_head
      output_depth: Size last dimension of the final output
      num_heads: Number of attention heads
      bias_mask: Masking tensor to prevent connections to future elements
      dropout: Dropout probability (Should be non-zero only during training)
    """
    super(Attention, self).__init__()
    # Checks borrowed from 
    # https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
    if total_key_depth % num_heads != 0:
      raise ValueError("Key depth (%d) must be divisible by the number of "
               "attention heads (%d)." % (total_key_depth, num_heads))
    if total_value_depth % num_heads != 0:
      raise ValueError("Value depth (%d) must be divisible by the number of "
               "attention heads (%d)." % (total_value_depth, num_heads))
      
    self.num_heads = num_heads
    #self.query_scale = 8**-0.5
    self.bias_mask = bias_mask
    
    # Key and query depth will be same
    self.query_linear = nn.Linear(input_depth, total_key_depth, bias=False)
    self.key_linear = nn.Linear(input_depth, total_key_depth, bias=False)
    self.value_linear = nn.Linear(input_depth, total_value_depth, bias=False)
    self.output_linear = nn.Linear(total_value_depth, output_depth, bias=False)
    
    self.dropout = nn.Dropout(dropout)
  
  def _split_heads(self, x):
    """
    Split x such to add an extra num_heads dimension
    Input:
      x: a Tensor with shape [batch_size, seq_length, depth]
    Returns:
      A Tensor with shape [batch_size, num_heads, seq_length, depth/num_heads]
    """
    if len(x.shape) != 3:
      raise ValueError("x must have rank 3")
    shape = x.shape
    return x.view(shape[0], shape[1], self.num_heads, shape[2]//self.num_heads).permute(0, 2, 1, 3)
  
  def _merge_heads(self, x):
    """
    Merge the extra num_heads into the last dimension
    Input:
      x: a Tensor with shape [batch_size, num_heads, seq_length, depth/num_heads]
    Returns:
      A Tensor with shape [batch_size, seq_length, depth]
    """
    if len(x.shape) != 4:
      raise ValueError("x must have rank 4")
    shape = x.shape
    return x.permute(0, 2, 1, 3).contiguous().view(shape[0], shape[2], shape[3]*self.num_heads)
    
  def forward(self, queries, keys, values):
    
    # Do a linear for each component
    queries = queries
    keys = keys
    values = values
    
    # Split into multiple heads
    queries = self._split_heads(queries)
    keys = self._split_heads(keys)
    values = self._split_heads(values)

    # Combine queries and keys
    logits = torch.matmul(queries, keys.permute(0, 1, 3, 2))

    # Convert to probabilites
    weights = nn.functional.softmax(logits, dim=-1)
    
    # Combine with values to get context
    contexts = torch.matmul(weights, values)
    
    # Merge heads
    contexts = self._merge_heads(contexts)

    return contexts

In [ ]:
def get_context(input_l2r, input_r2l, window = 3):
  batch_size = input_l2r.size(0)
  input_seq_len = input_l2r.size(1)
  pad_window = torch.zeros(input_l2r.size(0), window, input_l2r.size(2))
  if using_GPU:
    pad_window = pad_window.cuda()
  input_l2r = torch.cat([pad_window, input_l2r], 1)
  input_r2l = torch.cat([input_r2l, pad_window], 1)
  context_l2r = input_l2r.unsqueeze(1).expand(batch_size, input_seq_len, input_l2r.size(1), 
                          input_l2r.size(2)).contiguous().view(batch_size*input_seq_len,
                                             input_l2r.size(1),input_l2r.size(2))
  context_r2l = input_r2l.unsqueeze(1).expand(batch_size, input_seq_len, input_r2l.size(1), 
                          input_r2l.size(2)).contiguous().view(batch_size*input_seq_len,
                                             input_r2l.size(1),input_r2l.size(2))
  mask_eye = torch.from_numpy(np.eye(window, dtype=int)).type(torch.FloatTensor)

  mask_tensor_l2r = torch.cat([mask_eye, torch.zeros(context_l2r.size(1)-mask_eye.size(0),
                                 mask_eye.size(1)).type(torch.FloatTensor)], 0)
  mask_tensor_r2l = torch.cat([torch.cat([torch.zeros(1, mask_eye.size(-1)).type(torch.FloatTensor), 
                          mask_eye], 0), torch.zeros(context_l2r.size(1)-mask_eye.size(0)-1,
                           mask_eye.size(1)).type(torch.FloatTensor)], 0)

  # forming masks based on the window size
  for i in range(1, context_l2r.size(1)-window):
    mask_tensor_l2r = torch.cat([mask_tensor_l2r, 
                       torch.zeros(i, mask_eye.size(1)).type(torch.FloatTensor)], 0)
    mask_tensor_l2r = torch.cat([mask_tensor_l2r,
                      mask_eye], 0)
    mask_tensor_l2r = torch.cat([mask_tensor_l2r,
                      torch.zeros(context_l2r.size(1)-i-mask_eye.size(0), 
                            mask_eye.size(1)).type(torch.FloatTensor)], 0)

    mask_tensor_r2l = torch.cat([mask_tensor_r2l, 
                       torch.zeros(i+1, mask_eye.size(1)).type(torch.FloatTensor)], 0)
    mask_tensor_r2l = torch.cat([mask_tensor_r2l,
                      mask_eye], 0)
    mask_tensor_r2l = torch.cat([mask_tensor_r2l,
                      torch.zeros(context_r2l.size(1)-i-1-mask_eye.size(0), 
                            mask_eye.size(1)).type(torch.FloatTensor)], 0)
  mask_l2r = mask_tensor_l2r.unsqueeze(0).expand(batch_size, mask_tensor_l2r.size(0), mask_tensor_l2r.size(1))
  mask_r2l = mask_tensor_r2l.unsqueeze(0).expand(batch_size, mask_tensor_r2l.size(0), mask_tensor_r2l.size(1))

  mask_l2r = Variable(mask_l2r.contiguous().view(context_l2r.size(0), context_l2r.size(1), -1), 
              requires_grad = False)
  mask_r2l = Variable(mask_r2l.contiguous().view(context_r2l.size(0), context_r2l.size(1), -1), 
              requires_grad = False)
  if using_GPU:
    mask_l2r = mask_l2r.cuda()
    mask_r2l = mask_r2l.cuda()

  maskted_context_l2r = torch.bmm(context_l2r.permute(0,2,1),mask_l2r).permute(0,2,1)
  maskted_context_r2l = torch.bmm(context_r2l.permute(0,2,1),mask_r2l).permute(0,2,1)
    
  return maskted_context_l2r, maskted_context_r2l

In [ ]:
def get_query(input):
  query = input.unsqueeze(2).view(input.size(0)*input.size(1), 1, input.size(2))
  return query

## RNN model

In [ ]:
class BiLSTMAtt(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, num_layers, dropout_lstm_input, 
          dropout_fc_input, dropout_rnn, bidir=True, target_size=2):
    # Always call superclass first (nn.Module)
    super(BiLSTMAtt, self).__init__()

    # set RNN model
    # dropout_rnn = 0 when num_layers = 1
    self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                num_layers=num_layers, dropout=dropout_rnn, 
                batch_first=True, bidirectional=bidir)
    # set dropout layer
    self.dropout_on_input_to_LSTM = nn.Dropout(dropout_lstm_input)
    self.dropout_on_input_to_linear_layer = nn.Dropout(dropout_fc_input)

    # set up the final transform to a distribution over classes.
    direc = 2 if bidir else 1
    self.fc = nn.Linear(hidden_dim * direc * 2, target_size)

    # set up attenion layer
    self.attention = Attention(input_depth=hidden_dim, total_key_depth=64, 
                   total_value_depth=64, output_depth=hidden_dim, 
                   num_heads=16, bias_mask=None, dropout=0.0)

  # get_lstm_features takes an batch of inputs and a list of ints
  # List of int with the unpadded length of each example in the batch.
  # input is of shape (batch_size, sequence_length)
  def forward(self, inputs, lengths):
    # 1. run LSTM
    # apply dropout to the input
    # Shape of inputs: (batch_size, sequence_length, embedding_dim)
    embedded_input = self.dropout_on_input_to_LSTM(inputs)
    # Sort the embedded inputs by decreasing order of input length.
    # sorted_input shape: (batch_size, sequence_length, embedding_dim)
    (sorted_input, sorted_lengths, input_unsort_indices, _) = sort_batch_by_length(embedded_input, lengths)
    # Pack the sorted inputs with pack_padded_sequence.
    packed_input = pack_padded_sequence(sorted_input, sorted_lengths.data.tolist(), batch_first=True)
    # Run the input through the RNN.
    packed_sorted_output, _ = self.rnn(packed_input)
    # Unpack (pad) the input with pad_packed_sequence
    # Shape: (batch_size, sequence_length, hidden_dim)
    sorted_output, _ = pad_packed_sequence(packed_sorted_output, batch_first=True)
    # Re-sort the packed sequence to restore the initial ordering
    # Shape: (batch_size, sequence_length, hidden_dim)
    lstm_output = sorted_output[input_unsort_indices]

    # 2. run attention layer
    # set the dimension of attention vector as hidden dimension
    att_vec_dim = int(lstm_output.size(2)/2)
    # split the last dimension of lstm_output into 2 parts equally
    # Shape: [batch_size, sequence_length, hidden_dim]
    l2r = lstm_output[:, :, :att_vec_dim]
    r2l = lstm_output[:, :, att_vec_dim:]
    # get query for each part
    query_l2r = get_query(lstm_output[:, :, :att_vec_dim])
    query_r2l = get_query(lstm_output[:, :, att_vec_dim:])
    # get context for each part
    context_l2r, context_r2l = get_context(lstm_output[:, :, :att_vec_dim], 
                         lstm_output[:, :, att_vec_dim:], 
                         window = 3)
    # run each part through attention layer to get two attention encodings
    # Shape: [batch_size, sequence_length, hidden_dim] 
    att_l2r = self.attention(query_l2r, context_l2r, context_l2r).view(
        lstm_output.size(0), lstm_output.size(1), -1)
    att_r2l = self.attention(query_r2l, context_r2l, context_r2l).view(
        lstm_output.size(0), lstm_output.size(1), -1)
    # concatenate these two attention encodings into one full attention encoding 
    # Shape: [batch_size, sequence_length, hidden_dim * 2]
    att = torch.cat([att_l2r, att_r2l], -1)    
    # concatenate LSTM layer output with the full attention encoding
    # Shape: [batch_size, sequence_length, hidden_dim * 4]
    attention_output = torch.cat([lstm_output, att], -1)
    
    # 3. run linear layer
    # apply dropout to input to the linear layer
    FC_input_encoding = self.dropout_on_input_to_linear_layer(attention_output)
    # run through the output projection to get scores for each of the classes.
    linear_output = self.fc(FC_input_encoding)
    # Normalize with log softmax
    output_distribution = F.log_softmax(linear_output, dim=-1)

    return output_distribution

# 3. Functions

## read data

In [ ]:
def get_raw_data(path):
  raw = []
  with open(path) as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
      raw.append(line)
  return raw  

In [ ]:
def get_PoS_tag(tag):
  ''' Tagset convertion '''
  if tag in ['a', 'ag', 'an', 'b', 'z', 'zg']:
    convert = 'ADJ'
  elif tag in ['p', 'f']:
    convert = 'ADP'
  elif tag in ['ad', 'd', 'dg', 't']:
    convert = 'ADV'
  elif tag in ['n', 'ng', 'nr', 'ns', 's', 'nz', 'g', 'j', 'nt', 'nx', 'tg']:
    convert = 'NOUN'
  elif tag in ['r', 'rg']:
    convert = 'PRNOUN'
  elif tag in ['v', 'vd', 'vg', 'vn']:
    convert = 'VERB'
  else:
    convert = 'X'
  return convert

In [ ]:
def get_PoS_tag_VUA(tag):
  ''' Tagset convertion '''
  if tag in ['ADJ']:
    convert = 'ADJ'
  elif tag in ['ADP']:
    convert = 'ADP'
  elif tag in ['ADV']:
    convert = 'ADV'
  elif tag in ['NOUN']:
    convert = 'NOUN'
  elif tag in ['PROPN', 'PROPN']:
    convert = 'PRNOUN'
  elif tag in ['VERB']:
    convert = 'VERB'
  else:
    convert = 'X'
  return convert

In [ ]:
def predict_dataset(dataset):
  '''
  get dataset for predict
  :param dataset: original dataset 
            index, sent, pos, met, genre, orignial tokens, bert tokens,
            bert toekns in original style, BIO tags
  :return predict: for predict
            original tokenized sentence, metaphor labels, POS tags 
  '''
  #  original tokenized sentence, metaphor labels, POS tags
  predict = [[ast.literal_eval(row[5]),
          ast.literal_eval(row[3]),
          [get_PoS_tag(tag) for tag in ast.literal_eval(row[2])]] 
          for row in dataset]
  return predict

In [ ]:
def predict_dataset_VUA(dataset):
  '''
  get dataset for predict
  :param dataset: original dataset 
            index, sent, pos, met, genre, orignial tokens, bert tokens,
            bert toekns in original style, BIO tags
  :return predict: for predict
            original tokenized sentence, metaphor labels, POS tags 
  '''
  #  original tokenized sentence, metaphor labels, POS tags
  predict = [[ast.literal_eval(row[5]),
          ast.literal_eval(row[3]),
          [get_PoS_tag_VUA(tag) for tag in ast.literal_eval(row[2])]] 
          for row in dataset]
  return predict

In [ ]:
def get_pos2idx_idx2pos(pos_set):
  pos_set = pos_set
  idx2pos = dict(enumerate(pos_set))
  pos2idx = dict([(v, k) for (k, v) in idx2pos.items()])
  return pos2idx, idx2pos

In [ ]:
def get_token2idx_idx2token(vocab):
  vocab = vocab
  idx2token = dict(enumerate(vocab))
  token2idx = dict([(v, k) for (k, v) in idx2token.items()])
  return token2idx, idx2token

In [ ]:
def countX(lst, x):
  ''' count the number of an element in a list'''
  count = 0
  for ele in lst:
    if (ele == x):
      count = count + 1
  return count

In [ ]:
def describe(dataset):
  '''
  :param dataset: a formatted list of five-item lists
             - sentences
             - metaphor sequence
             - pos sequence
             - genre
             - split
  :return: print description of the dataset
        - first 5 lines
        - sentence length (mean and SD) 
          # Considering the amount of data, we directly use mean and SD without distribution check
        - literal & metaphor usages for each pos tag (1 for metaphor)
        - POS tag count
        - genre count
  '''
  # print the first and last 5 lines
  print('>>> 1. Size of the dataset')
  print(['sentences', 'pos_seq', 'met_seq', 'genre', 'split']) 
  print('Size: ', len(dataset))

  # print mean and SD of sentence length
  print('\n>>> 2. Sentence length') 
  sentences = [line[0] for line in dataset]
  print('Mean\t{:.2f}'.format(np.mean([len(sentence) for sentence in sentences])))
  print('SD\t{:.2f}'.format(np.std([len(sentence) for sentence in sentences])))

  # print metaphorical label porpotion
  print('\n>>> 3. Metaphor count (1 for metaphor):')
  ## pos and tag list
  met = [line[1] for line in dataset]
  pos = [line[2] for line in dataset]
  ## into a list
  pos2met = []
  for i in list(zip(pos, met)):
    pos2met.extend(list(zip(i[0], i[1])))
  ## count
  for i in sorted(set(pos2met)):
    print('{}\t{}'.format(i, countX(pos2met, i)))
  
  # print pos tag numbers
  print('\n>>> 4. POS tag count:')
  vocab = sum([row[0] for row in dataset], [])
  pos_list = []
  for pos_seq in pos:
    pos_list.extend(pos_seq)
  ## count
  for i in sorted(set(pos_list)):
    print('{}\t{}\t{:.2%}'.format(i, countX(pos_list, i), countX(pos_list, i)/len(vocab)))

In [ ]:
def check_genre(dataset):
  genres = [line[4] for line in dataset]
  for i in sorted(set(genres)):
    print('{}: {}, {:.2%}'.format(i, countX(genres, i), countX(genres, i)/len(dataset)))

## get iterators

In [ ]:
def embed_sequence(sequence, elmo, FT):
  '''
  :param sequence: a single string, the original sequence
  :param Bert: a h5 file, pretrained Bert
          group key: a sentence   
          group value: pretrained sentence embeddings type: numpy array
  :param FT: a h5 file, pretrained FastText
         group key: a sentence
         group value: pretrained sentence embeddings type: numpy array
  :return: a concentrated embedding with glove, pretrained and suffix
        type: numpy array
  '''
  sentence = ''.join(sequence)
  # 1. embed the sentence by bert vectors
  elmo_part = elmo[sentence]
  assert (elmo_part.shape == (len(sequence), 1024))

  # 2. embed the sentence by bert vectors
  FT_part = FT[sentence]
  assert (FT_part.shape == (len(sequence), 300))

  # 3. concatenate
  result = np.concatenate((elmo_part, FT_part), axis=1)

  return result

In [ ]:
def get_dataloader(dataset, elmo, FT, batch_size, shuffle):
  '''
  :param: dataset: a splited dataset, train, test, or val
  :param: bert: pretrained sentence embeddings
  :param batch_size: size of batches
  :param: shuffle: a boolean value (True or False)
  :return: split_dataloader
  '''
  # embedded_split: embedded_sentence, indexed pos sequences, indexed tag sequences
  f = lambda x: [token2idx[token] for token in x]
  embedded_split = []
  for example in tqdm(dataset):
    embedded_sentence = embed_sequence(example[0], elmo, FT) # embedded sentence
    indexed_pos_seq = [pos2idx[pos] for pos in example[2]] # indexed pos sequences
    indexed_token_seq = [token2idx_psu[token] for token in example[0]]
    labels = example[1] # metaphor labels
    embedded_split.append([embedded_sentence, indexed_pos_seq, labels, indexed_token_seq])

  # separate the input (embedded_sequence) and labels in the indexed train sets.
  split_dataset = MetaphorDataset([example[0] for example in embedded_split],
                    [example[1] for example in embedded_split],
                    [example[2] for example in embedded_split],
                    [example[3] for example in embedded_split])
  # set up a DataLoader
  split_dataloader = data.DataLoader(dataset=split_dataset, batch_size=batch_size, 
                      shuffle=shuffle, collate_fn=MetaphorDataset.collate_fn)
  return split_dataloader

In [ ]:
def embed_sequence_VUA(sequence, elmo, FT):
  '''
  :param sequence: a single string, the original sequence
  :param Bert: a h5 file, pretrained Bert
          group key: a sentence   
          group value: pretrained sentence embeddings type: numpy array
  :param FT: a h5 file, pretrained FastText
         group key: a sentence
         group value: pretrained sentence embeddings type: numpy array
  :return: a concentrated embedding with glove, pretrained and suffix
        type: numpy array
  '''
  sentence = ' '.join(sequence)
  # 1. embed the sentence by bert vectors
  elmo_part = elmo[sentence]
  assert (elmo_part.shape == (len(sequence), 1024))

  # 2. embed the sentence by bert vectors
  FT_part = FT[sentence]
  assert (FT_part.shape == (len(sequence), 300))

  # 3. concatenate
  result = np.concatenate((elmo_part, FT_part), axis=1)

  return result

In [ ]:
def get_dataloader_VUA(dataset, elmo, FT, batch_size, shuffle):
  '''
  :param: dataset: a splited dataset, train, test, or val
  :param: bert: pretrained sentence embeddings
  :param batch_size: size of batches
  :param: shuffle: a boolean value (True or False)
  :return: split_dataloader
  '''
  # embedded_split: embedded_sentence, indexed pos sequences, indexed tag sequences
  f = lambda x: [token2idx[token] for token in x]
  embedded_split = []
  for example in tqdm(dataset):
    embedded_sentence = embed_sequence_VUA(example[0], elmo, FT) # embedded sentence
    indexed_pos_seq = [pos2idx[pos] for pos in example[2]] # indexed pos sequences
    indexed_token_seq = [token2idx_vua[token] for token in example[0]]
    labels = example[1] # metaphor labels
    embedded_split.append([embedded_sentence, indexed_pos_seq, labels, indexed_token_seq])

  # separate the input (embedded_sequence) and labels in the indexed train sets.
  split_dataset = MetaphorDataset([example[0] for example in embedded_split],
                    [example[1] for example in embedded_split],
                    [example[2] for example in embedded_split],
                    [example[3] for example in embedded_split])
  # set up a DataLoader
  split_dataloader = data.DataLoader(dataset=split_dataset, batch_size=batch_size, 
                      shuffle=shuffle, collate_fn=MetaphorDataset.collate_fn)
  return split_dataloader

In [ ]:
# Make sure to subclass torch.utils.data.Dataset
# To convert a split dataset into batches to create iterator
class MetaphorDataset(data.Dataset):
  def __init__(self, embedded_text, pos_seqs, tag_seqs, tokenized_text):
    '''
    :param embedded_text: sentence embeddings
    :param tokenized_text: indexed tokenized sentece 
    :param pos_seqs: indexed pos sequences
    :param tag_seqs: indexed metaphor label sequences
    '''
    if len(embedded_text) != len(tag_seqs):
      raise ValueError('Differing number of sentences and tags!')
    
    # A list of h5py File results, available for convert into tensor, numpy array, list etc.
    # where each inner numpy arrays is sequence_length * embed_dim
    self.embedded_text = embedded_text
    # A list of indexed pos sequences
    # where each inner list is the indexed pos tags for the sentence at the corresponding index.
    self.pos_seqs = pos_seqs
    # A list of indexed BIO tags
    # where each inner list is the tag for the sentence at the corresponding index.
    self.tag_seqs = tag_seqs
    # A list of indexed tokenizations
    # where each inner list is the tags for tokens in bert tokenized sentece at the corresponing index
    self.tokenization = tokenized_text

  def __getitem__(self, idx):
    '''
    Return the Dataset example at index `idx`.
    '''
    example_pos_seq = self.pos_seqs[idx]
    example_text = self.embedded_text[idx]
    example_length = example_text.shape[0]
    example_tag_seq = self.tag_seqs[idx]
    example_tokenization = self.tokenization[idx]
    assert (example_length == len(example_pos_seq))
    assert (example_length == len(example_tag_seq))
    assert (example_length == len(example_tokenization))
    return example_pos_seq, example_text, example_length, example_tag_seq, example_tokenization
  
  def __len__(self):
    '''
    Return the number of examples in the Dataset.
    '''
    return len(self.tag_seqs)

  @staticmethod
  def collate_fn(batch):
    '''
    Given a list of examples (each from __getitem__), combine them to form a single batch by padding.
    
    Returns:
    -------
    batch_pos_seqs: list
      A list of list: each inner list is a variable-length list of sublists of indexed pos tags
    batch_padded_example_text: LongTensor
      LongTensor of shape (batch_size, longest_sequence_length) with the
      padded text for each example in the batch.
    length: LongTensor
      LongTensor of shape (batch_size,) with the unpadded length of the example.
    example_tag: LongTensor
      LongTensor of shape (batch_size,) with the tag of the example.
    '''
    batch_pos_seqs = []
    batch_padded_example_text = []
    batch_lengths = []
    batch_padded_tags = []
    batch_tokenization = []
    
    # Get the length of the longest sequence in the batch
    max_length = -1
    for _, _, _, tag, _ in batch:
      if len(tag) > max_length:
        max_length = len(tag)
    
    # Iterate over each example in the batch
    for pos, text, length, tag, tokenization in batch:
      # Unpack the example (returned from __getitem__)
      
      # append the pos_sequence to our batch
      batch_pos_seqs.append(pos)  

      # Fill shorter sentences by padding

      ## Amount to pad is length of longest example - length of this example. 
      amount_to_pad = max_length - length

      ## - fill tokenization by 0 ([PAD])
      tokenization = tokenization + [0] * amount_to_pad
      ## append the padded tokenizaiton to our batch
      batch_tokenization.append(tokenization)

      ## - fill sentence embedding by torch.zeros of shape (amount_to_pad,), converted to LongTensor
      pad_tensor = torch.zeros(amount_to_pad, text.shape[1])

      ## Append the pad_tensor to the example_text tensor.
      ## Shape of padded_example_text: (padded_length, embeding_dim)
      ## top part is the original text numpy,
      ## and the bottom part is the 0 padded tensors

      ## text embedding from the batch is a h5py File results 
      ## convert it to a torch.FloatTensor first to keep in same type with pad_tensor
      text = torch.Tensor(text)
      ## if we don't perform the conersion as below
      ## then here we can't concatenate 
      ## because tensor.cat requires two arguments to be the same types
      padded_example_text = torch.cat((text, pad_tensor), dim=0)
      ## append the padded example to our batch
      batch_padded_example_text.append(padded_example_text)

      # append the length to our batch
      batch_lengths.append(length)

      # Fill shorter pos sequences by padding
      # similar to what we did on shorter sentences
      # as tags from __getitem__ is a list, here we first fill with 0s in the list
      # then convert it to torch.LongTensor at the end
      padded_example_tag = tag + [0] * amount_to_pad
      # append the padded tag sequence to our batch
      batch_padded_tags.append(padded_example_tag)
    
    # Stack the list of LongTensors into a single LongTensor
    return (batch_pos_seqs,
        torch.stack(batch_padded_example_text),
        torch.LongTensor(batch_lengths),
        torch.LongTensor(batch_padded_tags),
        torch.LongTensor(batch_tokenization))

## evaluate

In [ ]:
def get_batch_predictions(pos_seqs, tag_seqs, predict_seqs):
  '''
  :param pos_seqs: A list of sublists of ints, indexed pos tags
  :param tag_seqs: LongTensor of shape (batch_size,) with the padded length
  :param predict_seqs: A list of ints, 
  :return: pos_tags: a list of ints, indexed pos tags 
       gold_labels:  a list of ints, with the orignial tokenization length
       predict_labels: a list of ints, with the orignial tokenization length
       # number of sublists: number of example in that batch
  '''
  
  
  # first put the tensor on CPU and then convert them into lists
  predict_seqs = torch.max(predict_seqs.data, 2)[1]
  tag_seqs = tag_seqs.cpu().numpy().tolist()
  assert (len(pos_seqs) == len(tag_seqs))
  assert (len(pos_seqs) == predict_seqs.shape[0])

  pos_tags = []
  gold_labels = []
  predict_labels = []
  for i in range(len(pos_seqs)):
    # convert pos seq to pos tag
    pos_tag = pos_seqs[i]
    # convert the BIO tags into metaphor labels
    gold_label = tag_seqs[i][:len(pos_seqs[i])]
    predict_label = predict_seqs[i][:len(pos_seqs[i])]
    assert (len(pos_tag) == len(gold_label))
    assert (len(pos_tag) == len(predict_label))

    # concatenate these results respectively into a single list
    pos_tags.append(pos_tag)
    gold_labels.append(gold_label)
    predict_labels.append(predict_label)
  
  return (pos_tags, gold_labels, predict_labels) 

In [ ]:
def update_confusion_matrix(matrix, predictions, labels, pos_seqs):
  '''
  update the confusion matrix based on the given batch

  :param matrix: a 3D numpy array of shape (#pos_tags, 2, 2)
  :param predictions: a numpy array of shape (batch_size, max_seq_len)
  :param labels: a numpy array of shape (batch_size, max_seq_len)
  :param lengths: a numpy array of shape (batch_size)
  :param pos_seqs: a list of variable-length indexed pos sequence
  :param idx2pos: a dictionary: int --> pos tag
  :return: the updated matrix
  '''
  for i in range(len(pos_seqs)):
    indexed_pos_sequence = pos_seqs[i]
    prediction = predictions[i]
    label = labels[i]
    for j in range(len(indexed_pos_sequence)):
      indexed_pos = indexed_pos_sequence[j]
      p = prediction[j]
      l = label[j]
      matrix[indexed_pos][p][l] += 1
  return matrix

In [ ]:
def print_info(matrix, idx2pos):
  '''
  Prints the precision, recall, f1, and accuracy for each pos tag
  Assume that the confusion matrix is implicitly mapped with the idx2pos
  i.e. row 0 in confusion matrix is for the pos tag mapped by int 0 in idx2pos

  :param matrix: a confusion matrix of shape (#pos_tags, 2, 2)
  :param idx2pos: idx2pos: a dictionary: int --> pos tag
  :return: a matrix (#allpostags, 4) each row is the PRFA performance for a pos tag
  '''
  result = []
  print('\t\t\t\tPrecision\tRecall\t\tF1\t\tAccuracy\tMatrix')
  for idx in range(len(idx2pos)):
    pos_tag = idx2pos[idx]
    grid = matrix[idx]
    precision = 100 * grid[1, 1] / np.sum(grid[1])
    recall = 100 * grid[1, 1] / np.sum(grid[:, 1])
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = 100 * (grid[1, 1] + grid[0, 0]) / np.sum(grid)
    print('- PRFA performance for {}:\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}'.format(pos_tag, precision, recall, f1, accuracy, grid.tolist()))
    result.append([precision, recall, f1, accuracy])
  return np.array(result)

In [ ]:
def evaluate(model, evaluation_dataloader, evaluation_dataset):
  '''
  Evaluate the model on the given evaluation_dataloader
  :param model:
  :param evaluation_dataloader:
  :param evaluation_dataset:
  :return:
  - average_eval_loss
  - a matrix (#allpostags, 4) 
    each row is the PRFA performance for a pos tag
  '''
  # Set model to eval mode, which turns off dropout.
  model.eval()

  total_eval_loss = 0
  confusion_matrix = np.zeros((len(idx2pos), 2, 2))
  predictions = []

  for (eval_pos, eval_text, eval_lengths, eval_tags, eval_tokenizations) in tqdm(evaluation_dataloader):
    eval_text = Variable(eval_text)
    eval_lengths = Variable(eval_lengths)
    eval_tags = Variable(eval_tags)
    if using_GPU:
      eval_text = eval_text.cuda()
      eval_lengths = eval_lengths.cuda()
      eval_tags = eval_tags.cuda()
    
    # calculated training loss for this batch
    eval_prediction = model(eval_text, eval_lengths)
    batch_loss = loss_criterion(eval_prediction.view(-1, 2), eval_tags.view(-1))
    total_eval_loss += batch_loss

    # get pos tags, gold labels and prediced labels
    batch_pos, batch_gold, batch_predict = get_batch_predictions(eval_pos, eval_tags, eval_prediction)
    confusion_matrix = update_confusion_matrix(confusion_matrix, batch_predict, batch_gold, batch_pos)
    predictions.extend(batch_predict)

  average_eval_loss = total_eval_loss / evaluation_dataloader.__len__()

  # Set the model back to train mode, which activates dropout again.
  model.train()
  assert (len(predictions) == len(evaluation_dataset)) 

  return average_eval_loss, predictions, print_info(confusion_matrix, idx2pos)

In [ ]:
def get_performance_test(target_path):
  '''
  Read the test data and predictions
  Prints the performance of LSTM sequence model on based on genre
  Prints the performance of LSTM sequence model on regardless of genre
  :param target_path: the file with prediction

  :return: the averaged performance across genre
  '''
  # get the prediction from LSTM sequence model
  ID2sen_labelseq = {}  # ID tuple --> [genre, label_sequence, pred_sequence]
  with open(target_path) as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
      ID2sen_labelseq[line[0]] = [line[4], 
                      ast.literal_eval(line[3]), 
                      ast.literal_eval(line[-1])]
  # compute confusion_matrix
  genres = ['News', 'Fiction', 'Academic']
  confusion_matrix = np.zeros((4, 2, 2))
  for ID in ID2sen_labelseq:
    genre, label_sequence, pred_sequence = ID2sen_labelseq[ID]
    for i in range(len(label_sequence)):
      pred = pred_sequence[i]
      label = label_sequence[i]
      genre_idx = genres.index(genre)
      confusion_matrix[genre_idx][pred][label] += 1

  print('Genre-specific Performance:')
  print('\t\tPrecision\tRecall\t\tF1\t\tAccuracy\tMatrix')
  avg_performance = []
  for i in range(len(genres)):
    precision = 100 * confusion_matrix[i, 1, 1] / np.sum(confusion_matrix[i, 1])
    recall = 100 * confusion_matrix[i, 1, 1] / np.sum(confusion_matrix[i, :, 1])
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = 100 * (confusion_matrix[i, 1, 1] + confusion_matrix[i, 0, 0]) / np.sum(confusion_matrix[i])
    if i == 0:
      print('- {}:\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}'.format(genres[i], precision, recall, f1, accuracy, confusion_matrix[i].tolist()))
    else:
      print('- {}:\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}'.format(genres[i], precision, recall, f1, accuracy, confusion_matrix[i].tolist()))
    avg_performance.append([precision, recall, f1, accuracy])
  avg_performance = np.array(avg_performance)

  print('\nGeneral Performance')
  confusion_matrix = confusion_matrix.sum(axis=0)
  precision = 100 * confusion_matrix[1, 1] / np.sum(confusion_matrix[1])
  recall = 100 * confusion_matrix[1, 1] / np.sum(confusion_matrix[:, 1])
  f1 = 2 * precision * recall / (precision + recall)
  accuracy = 100 * (confusion_matrix[1, 1] + confusion_matrix[0, 0]) / np.sum(confusion_matrix)
  print('Precision\tRecall\t\tF1\t\tAccuracy\tMatrix')
  print('{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}\n'.format(precision, recall, f1, accuracy, confusion_matrix.tolist()))

  return avg_performance.mean(0)


In [ ]:
def get_performance_pos(target_path, pos):
  '''
  Similar to get_performance_VUA_test
  In this function we aim to see the performance on a certain word class
  Prints the performance of LSTM sequence model on based on genre
  Prints the performance of LSTM sequence model on regardless of genre
  :param target_path: the file with prediction
              ['sent_index', 'sent_txt', 'pos_seq', 'metaphor_seq', 'genre', 'sent_txt_tokenized',  'sent_bert_tokens', 'sent_txt_tokenized_bert', 'BIO_seq', 'prediction']
  :param pos: the pos to investigate
  '''

  # get the prediction from LSTM sequence model
  ID2info = {}  # (ID, pos_idx) --> [pos_idx, genre, gold label, predict label]
  with open(target_path) as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
      pos_index = [i[0] for i in enumerate(ast.literal_eval(line[2])) if i[1] == pos]
      for index in pos_index:
        ID2info[(line[0], index)] = [index,
                         line[4], 
                         ast.literal_eval(line[3])[index], 
                         ast.literal_eval(line[-1])[index]]
  # compute confusion_matrix
  predictions = []
  genres = ['News', 'Fiction', 'Academic']
  confusion_matrix = np.zeros((4, 2, 2))
  for (ID, index) in ID2info:
    index, genre, gold, pred = ID2info[(ID, index)]
    predictions.append(pred)
    genre_idx = genres.index(genre)
    confusion_matrix[genre_idx][pred][gold] += 1
  assert (np.sum(confusion_matrix) == len(ID2info))
  
  print(pos)
  print('Genre-specific Performance:')
  print('\t\tPrecision\tRecall\t\tF1\t\tAccuracy\tMatrix')
  avg_performance = []
  for i in range(len(genres)):
    precision = 100 * confusion_matrix[i, 1, 1] / np.sum(confusion_matrix[i, 1])
    recall = 100 * confusion_matrix[i, 1, 1] / np.sum(confusion_matrix[i, :, 1])
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = 100 * (confusion_matrix[i, 1, 1] + confusion_matrix[i, 0, 0]) / np.sum(confusion_matrix[i])
    if i == 0:
      print('- {}:\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}'.format(genres[i], precision, recall, f1, accuracy, confusion_matrix[i].tolist()))
    else:
      print('- {}:\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}'.format(genres[i], precision, recall, f1, accuracy, confusion_matrix[i].tolist()))    
    avg_performance.append([precision, recall, f1, accuracy])
  avg_performance = np.array(avg_performance)

  print('\nGeneral Performance')
  confusion_matrix = confusion_matrix.sum(axis=0)
  precision = 100 * confusion_matrix[1, 1] / np.sum(confusion_matrix[1])
  recall = 100 * confusion_matrix[1, 1] / np.sum(confusion_matrix[:, 1])
  f1 = 2 * precision * recall / (precision + recall)
  accuracy = 100 * (confusion_matrix[1, 1] + confusion_matrix[0, 0]) / np.sum(confusion_matrix)
  print('Precision\tRecall\t\tF1\t\tAccuracy\tMatrix')
  print('{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{}\n'.format(precision, recall, f1, accuracy, confusion_matrix.tolist()))

  return avg_performance.mean(0)

# 4. Prepare data

## preprocess data 

In [ ]:
# data file path
psu = '/content/drive/MyDrive/Metaphor Detection/data/PSUCMC_sequence_new.csv'
psu_train = '/content/drive/MyDrive/Metaphor Detection/data/PSUCMC_sequence_train.csv'
psu_test = '/content/drive/MyDrive/Metaphor Detection/data/PSUCMC_sequence_test.csv'
psu_val = '/content/drive/MyDrive/Metaphor Detection/data/PSUCMC_sequence_val.csv'

In [ ]:
vua = '/content/drive/MyDrive/Metaphor Detection/data/VUA_sequence_new.csv'
vua_train = '/content/drive/MyDrive/Metaphor Detection/data/VUA_sequence_train.csv'
vua_test = '/content/drive/MyDrive/Metaphor Detection/data/VUA_sequence_test.csv'
vua_val = '/content/drive/MyDrive/Metaphor Detection/data/VUA_sequence_val.csv'

In [ ]:
# get raw data
raw_psu = get_raw_data(psu)
raw_vua = get_raw_data(vua)
raw_train = get_raw_data(vua_train)
raw_test = get_raw_data(psu_test)
raw_val = get_raw_data(psu_val)

In [ ]:
# set to predict
pred_psu = predict_dataset(raw_psu)
pred_vua = predict_dataset_VUA(raw_vua)
pred_train = predict_dataset_VUA(raw_train)
pred_test = predict_dataset(raw_test)
pred_val = predict_dataset(raw_val)
print(f'Train/test/val split: {len(pred_train)}, {len(pred_test)}, {len(pred_val)}')

Train/test/val split: 6323, 345, 345


In [ ]:
# describe dataset
print('\n========== PSUCMC ==========\n')
describe(pred_psu)
print('\n========== VUAMC ==========\n')
describe(pred_vua)
# check genre porpotion
print('\n-----------------------------------------------------')
print('\nGenre Ratio:')
print('\nall')
check_genre(raw_psu)
print('\ntrain')
check_genre(raw_train)
print('\ntest')
check_genre(raw_test)
print('\nval')
check_genre(raw_val)


========== PSUCMC ==========

>>> 1. Size of the dataset
['sentences', 'pos_seq', 'met_seq', 'genre', 'split']
Size:  1724

>>> 2. Sentence length
Mean	20.73
SD	13.93

>>> 3. Metaphor count (1 for metaphor):
('ADJ', 0)	1252
('ADJ', 1)	242
('ADP', 0)	1134
('ADP', 1)	692
('ADV', 0)	2650
('ADV', 1)	20
('NOUN', 0)	7767
('NOUN', 1)	570
('PRNOUN', 0)	1710
('PRNOUN', 1)	232
('VERB', 0)	5891
('VERB', 1)	1343
('X', 0)	12069
('X', 1)	175

>>> 4. POS tag count:
ADJ	1494	4.18%
ADP	1826	5.11%
ADV	2670	7.47%
NOUN	8337	23.32%
PRNOUN	1942	5.43%
VERB	7234	20.24%
X	12244	34.25%

========== VUAMC ==========

>>> 1. Size of the dataset
['sentences', 'pos_seq', 'met_seq', 'genre', 'split']
Size:  10567

>>> 2. Sentence length
Mean	19.44
SD	13.86

>>> 3. Metaphor count (1 for metaphor):
('ADJ', 0)	14851
('ADJ', 1)	2000
('ADP', 0)	16425
('ADP', 1)	6741
('ADV', 0)	11776
('ADV', 1)	819
('NOUN', 0)	30951
('NOUN', 1)	5010
('PRNOUN', 0)	9366
('PRNOUN', 1)	120
('VERB', 0)	30929
('VERB', 1)	7012
('X', 0)	67346
('X

In [ ]:
# pos2idx
pos_set = sorted(set(sum([row[2] for row in pred_psu], [])))
pos2idx, idx2pos = get_pos2idx_idx2pos(pos_set)

In [ ]:
pos_set

['ADJ', 'ADP', 'ADV', 'NOUN', 'PRNOUN', 'VERB', 'X']

In [ ]:
# token2idx
vocab_psu = sorted(list(set(sum([row[0] for row in pred_psu], []))))
# 0: [PAD], used to fill sentences to the max length in batch
vocab_psu.insert(0, '[PAD]') 
token2idx_psu, idx2token_psu = get_token2idx_idx2token(vocab_psu)
print(f'PSU vocab size: {len(vocab_psu)-1}')

PSU vocab size: 7124


In [ ]:
# token2idx
vocab_vua = sorted(list(set(sum([row[0] for row in pred_vua], []))))
# 0: [PAD], used to fill sentences to the max length in batch
vocab_vua.insert(0, '[PAD]') 
token2idx_vua, idx2token_vua = get_token2idx_idx2token(vocab_vua)
print(f'VUA vocab size: {len(vocab_vua)-1}')

VUA vocab size: 18655


## get dataloader

In [ ]:
# pretrained models
psu_elmo_path = '/content/drive/MyDrive/Metaphor Detection/embeddings/elmo/PSUCMC_elmo.h5'
psu_FT_path = '/content/drive/MyDrive/Metaphor Detection/embeddings/FastText/PSUCMC_FT.h5'
vua_elmo_path = '/content/drive/MyDrive/Metaphor Detection/embeddings/elmo/VUA_elmo.h5'
vua_FT_path = '/content/drive/MyDrive/Metaphor Detection/embeddings/FastText/VUA_FT.h5'

psu_elmo = h5py.File(psu_elmo_path)
psu_FT = h5py.File(psu_FT_path) 
vua_elmo = h5py.File(vua_elmo_path)
vua_FT = h5py.File(vua_FT_path) 

In [ ]:
# embed the datasets and get DataLoaders
# shuffle the train dataset to improve the reliabity of training model
# but do not shuffle test and validation set
# esp. when you want to print predictions out 
# if you shuffle them, then although you could get correct PRFA performance scores
# you cannot know which line the prediction belongs to 
train_dataloader = get_dataloader_VUA(dataset=pred_train, elmo=vua_elmo, FT=vua_FT, batch_size=10, shuffle=True)
test_dataloader = get_dataloader(dataset=pred_test, elmo=psu_elmo, FT=psu_FT, batch_size=10, shuffle=False)
val_dataloader = get_dataloader(dataset=pred_val, elmo=psu_elmo, FT=psu_FT, batch_size=10, shuffle=False)

# 5. Train 

## setting

In [ ]:
# Instantiate the model
# embedding_dim = elmo+FT
# activate dropout_rnn when num_layers != 1
# hidden_dim % 16 == 0
rnn_model = BiLSTMAtt(embedding_dim=1324, hidden_dim=256, num_layers=1, target_size=2,
            dropout_lstm_input=0.5, dropout_fc_input=0.5, dropout_rnn=0, bidir=True)
if using_GPU: 
  rnn_model = rnn_model.cuda()

In [ ]:
# Set up criterion for calculating loss
loss_criterion = nn.NLLLoss()

In [ ]:
# optimizer for updating the parameters
rnn_optimizer = torch.optim.AdamW(rnn_model.parameters(), lr=5e-3, eps=1e-10)

In [ ]:
# Number of epochs (passes through the dataset) to train the model for.
num_epochs = 15

## Train and evaluate on val

In [ ]:
train_loss = []
val_loss = []
performance_matrix = None

num_iter = 0
for epoch in range(num_epochs):
  print(">>> Starting epoch {}".format(epoch + 1))
  total_train_loss = 0
  for (example_pos, example_text, example_lengths, example_labels, example_tokenizations) in tqdm(train_dataloader):
    example_text = Variable(example_text)
    example_lengths = Variable(example_lengths)
    example_labels = Variable(example_labels)
    if using_GPU:
      example_text = example_text.cuda()
      example_lengths = example_lengths.cuda()
      example_labels = example_labels.cuda()
    
    # calculated training loss for this batch
    train_prediction = rnn_model(example_text, example_lengths)
    batch_loss = loss_criterion(train_prediction.view(-1, 2), example_labels.view(-1))
    total_train_loss += batch_loss
    rnn_optimizer.zero_grad()
    batch_loss.backward()
    rnn_optimizer.step()
    num_iter += 1

  average_train_loss = total_train_loss / train_dataloader.__len__()
  train_loss.append(average_train_loss)

  # Calculate validation and training set loss and accuracy every 104 gradient updates
  if num_iter % train_dataloader.__len__() == 0:
    # val
    print('Validation Set')
    average_val_loss, predictions, performance_matrix = evaluate(rnn_model, val_dataloader, pred_val)
    val_loss.append(average_val_loss)
    print('Iteration {}. Training Loss {:.4f}. Validation Loss {:.4f}.\n'.
           format(num_iter, torch.Tensor.log(average_train_loss), torch.Tensor.log(average_val_loss)))


>>> Starting epoch 1



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	14.29		1.82		3.23		79.73		[[235.0, 54.0], [6.0, 1.0]]
- PRFA performance for ADP:	15.38		1.60		2.90		60.12		[[200.0, 123.0], [11.0, 2.0]]
- PRFA performance for ADV:	0.00		0.00		nan		94.54		[[519.0, 6.0], [24.0, 0.0]]
- PRFA performance for NOUN:	6.38		2.42		3.51		90.31		[[1534.0, 121.0], [44.0, 3.0]]
- PRFA performance for PRNOUN:	nan		0.00		nan		86.90		[[345.0, 52.0], [0.0, 0.0]]
- PRFA performance for VERB:	22.58		2.45		4.42		79.00		[[1133.0, 279.0], [24.0, 7.0]]
- PRFA performance for X:	1.41		2.56		1.82		95.68		[[2390.0, 38.0], [70.0, 1.0]]
Iteration 633. Training Loss -1.5760. Validation Loss -1.4146.

>>> Starting epoch 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()



Validation Set


...


				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	6.45		3.64		4.65		72.30		[[212.0, 53.0], [29.0, 2.0]]
- PRFA performance for ADP:	44.19		15.20		22.62		61.31		[[187.0, 106.0], [24.0, 19.0]]
- PRFA performance for ADV:	0.87		16.67		1.65		78.32		[[429.0, 5.0], [114.0, 1.0]]
- PRFA performance for NOUN:	8.06		13.71		10.15		82.31		[[1384.0, 107.0], [194.0, 17.0]]
- PRFA performance for PRNOUN:	2.00		1.92		1.96		74.81		[[296.0, 51.0], [49.0, 1.0]]
- PRFA performance for VERB:	19.39		13.29		15.77		71.86		[[999.0, 248.0], [158.0, 38.0]]
- PRFA performance for X:	1.70		17.95		3.10		82.51		[[2055.0, 32.0], [405.0, 7.0]]
Iteration 8229. Training Loss -2.4275. Validation Loss -1.1187.

>>> Starting epoch 14



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	14.08		18.18		15.87		64.19		[[180.0, 45.0], [61.0, 10.0]]
- PRFA performance for ADP:	47.06		32.00		38.10		61.31		[[166.0, 85.0], [45.0, 40.0]]
- PRFA performance for ADV:	1.48		50.00		2.87		63.02		[[343.0, 3.0], [200.0, 3.0]]
- PRFA performance for NOUN:	7.33		25.00		11.33		71.50		[[1186.0, 93.0], [392.0, 31.0]]
- PRFA performance for PRNOUN:	5.49		9.62		6.99		66.50		[[259.0, 47.0], [86.0, 5.0]]
- PRFA performance for VERB:	22.48		34.27		27.15		63.55		[[819.0, 188.0], [338.0, 98.0]]
- PRFA performance for X:	1.31		25.64		2.48		68.59		[[1704.0, 29.0], [756.0, 10.0]]
Iteration 8862. Training Loss -2.4391. Validation Loss -0.8494.

>>> Starting epoch 15



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	6.52		5.45		5.94		67.91		[[198.0, 52.0], [43.0, 3.0]]
- PRFA performance for ADP:	44.83		20.80		28.42		61.01		[[179.0, 99.0], [32.0, 26.0]]
- PRFA performance for ADV:	1.56		33.33		2.99		76.32		[[417.0, 4.0], [126.0, 2.0]]
- PRFA performance for NOUN:	7.72		16.13		10.44		79.85		[[1339.0, 104.0], [239.0, 20.0]]
- PRFA performance for PRNOUN:	3.64		3.85		3.74		74.06		[[292.0, 50.0], [53.0, 2.0]]
- PRFA performance for VERB:	22.04		24.13		23.04		68.05		[[913.0, 217.0], [244.0, 69.0]]
- PRFA performance for X:	1.75		20.51		3.23		80.79		[[2011.0, 31.0], [449.0, 8.0]]
Iteration 9495. Training Loss -2.4469. Validation Loss -1.1109.



In [ ]:
# optimizer for updating the parameters
rnn_optimizer = torch.optim.AdamW(rnn_model.parameters(), lr=1e-4, eps=1e-10)

In [ ]:
train_loss = []
val_loss = []
performance_matrix = None

num_iter = 0
for epoch in range(10):
  print(">>> Starting epoch {}".format(epoch + 1))
  total_train_loss = 0
  for (example_pos, example_text, example_lengths, example_labels, example_tokenizations) in tqdm(train_dataloader):
    example_text = Variable(example_text)
    example_lengths = Variable(example_lengths)
    example_labels = Variable(example_labels)
    if using_GPU:
      example_text = example_text.cuda()
      example_lengths = example_lengths.cuda()
      example_labels = example_labels.cuda()
    
    # calculated training loss for this batch
    train_prediction = rnn_model(example_text, example_lengths)
    batch_loss = loss_criterion(train_prediction.view(-1, 2), example_labels.view(-1))
    total_train_loss += batch_loss
    rnn_optimizer.zero_grad()
    batch_loss.backward()
    rnn_optimizer.step()
    num_iter += 1

  average_train_loss = total_train_loss / train_dataloader.__len__()
  train_loss.append(average_train_loss)

  # Calculate validation and training set loss and accuracy every 104 gradient updates
  if num_iter % train_dataloader.__len__() == 0:
    # val
    print('Validation Set')
    average_val_loss, predictions, performance_matrix = evaluate(rnn_model, val_dataloader, pred_val)
    val_loss.append(average_val_loss)
    print('Iteration {}. Training Loss {:.4f}. Validation Loss {:.4f}.\n'.
           format(num_iter, torch.Tensor.log(average_train_loss), torch.Tensor.log(average_val_loss)))


>>> Starting epoch 1



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	6.25		5.45		5.83		67.23		[[196.0, 52.0], [45.0, 3.0]]
- PRFA performance for ADP:	46.97		24.80		32.46		61.61		[[176.0, 94.0], [35.0, 31.0]]
- PRFA performance for ADV:	1.40		33.33		2.68		73.59		[[402.0, 4.0], [141.0, 2.0]]
- PRFA performance for NOUN:	7.14		16.94		10.05		77.91		[[1305.0, 103.0], [273.0, 21.0]]
- PRFA performance for PRNOUN:	4.92		5.77		5.31		73.05		[[287.0, 49.0], [58.0, 3.0]]
- PRFA performance for VERB:	21.13		24.83		22.83		66.74		[[892.0, 215.0], [265.0, 71.0]]
- PRFA performance for X:	1.74		23.08		3.24		78.51		[[1953.0, 30.0], [507.0, 9.0]]
Iteration 633. Training Loss -2.4763. Validation Loss -1.0620.

>>> Starting epoch 2



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	5.88		5.45		5.66		66.22		[[193.0, 52.0], [48.0, 3.0]]
- PRFA performance for ADP:	49.28		27.20		35.05		62.50		[[176.0, 91.0], [35.0, 34.0]]
- PRFA performance for ADV:	1.38		33.33		2.65		73.22		[[400.0, 4.0], [143.0, 2.0]]
- PRFA performance for NOUN:	7.54		18.55		10.72		77.50		[[1296.0, 101.0], [282.0, 23.0]]
- PRFA performance for PRNOUN:	4.84		5.77		5.26		72.80		[[286.0, 49.0], [59.0, 3.0]]
- PRFA performance for VERB:	22.16		27.27		24.45		66.60		[[883.0, 208.0], [274.0, 78.0]]
- PRFA performance for X:	1.83		25.64		3.41		77.35		[[1923.0, 29.0], [537.0, 10.0]]
Iteration 1266. Training Loss -2.5044. Validation Loss -1.0362.

>>> Starting epoch 3


...


				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	6.78		7.27		7.02		64.19		[[186.0, 51.0], [55.0, 4.0]]
- PRFA performance for ADP:	48.15		31.20		37.86		61.90		[[169.0, 86.0], [42.0, 39.0]]
- PRFA performance for ADV:	1.88		50.00		3.61		70.86		[[386.0, 3.0], [157.0, 3.0]]
- PRFA performance for NOUN:	7.65		22.58		11.43		74.50		[[1240.0, 96.0], [338.0, 28.0]]
- PRFA performance for PRNOUN:	8.00		11.54		9.45		71.03		[[276.0, 46.0], [69.0, 6.0]]
- PRFA performance for VERB:	21.23		30.07		24.89		64.03		[[838.0, 200.0], [319.0, 86.0]]
- PRFA performance for X:	1.58		25.64		2.97		73.87		[[1836.0, 29.0], [624.0, 10.0]]
Iteration 5064. Training Loss -2.5517. Validation Loss -0.9606.

>>> Starting epoch 9



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	6.35		7.27		6.78		62.84		[[182.0, 51.0], [59.0, 4.0]]
- PRFA performance for ADP:	47.56		31.20		37.68		61.61		[[168.0, 86.0], [43.0, 39.0]]
- PRFA performance for ADV:	2.45		66.67		4.73		70.67		[[384.0, 2.0], [159.0, 4.0]]
- PRFA performance for NOUN:	7.53		22.58		11.29		74.15		[[1234.0, 96.0], [344.0, 28.0]]
- PRFA performance for PRNOUN:	7.69		11.54		9.23		70.28		[[273.0, 46.0], [72.0, 6.0]]
- PRFA performance for VERB:	21.12		30.42		24.93		63.69		[[832.0, 199.0], [325.0, 87.0]]
- PRFA performance for X:	1.68		28.21		3.18		73.19		[[1818.0, 28.0], [642.0, 11.0]]
Iteration 5697. Training Loss -2.5654. Validation Loss -0.9483.

>>> Starting epoch 10



Validation Set



				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	6.45		7.27		6.84		63.18		[[183.0, 51.0], [58.0, 4.0]]
- PRFA performance for ADP:	48.19		32.00		38.46		61.90		[[168.0, 85.0], [43.0, 40.0]]
- PRFA performance for ADV:	2.45		66.67		4.73		70.67		[[384.0, 2.0], [159.0, 4.0]]
- PRFA performance for NOUN:	7.65		23.39		11.53		73.85		[[1228.0, 95.0], [350.0, 29.0]]
- PRFA performance for PRNOUN:	7.69		11.54		9.23		70.28		[[273.0, 46.0], [72.0, 6.0]]
- PRFA performance for VERB:	20.81		30.42		24.72		63.27		[[826.0, 199.0], [331.0, 87.0]]
- PRFA performance for X:	1.67		28.21		3.15		72.95		[[1812.0, 28.0], [648.0, 11.0]]
Iteration 6330. Training Loss -2.5504. Validation Loss -0.9466.



# evaluate on test

In [ ]:
average_val_loss, predictions, performance_matrix = evaluate(rnn_model, test_dataloader, pred_test)


				Precision	Recall		F1		Accuracy	Matrix
- PRFA performance for ADJ:	15.79		24.49		19.20		64.06		[[168.0, 37.0], [64.0, 12.0]]
- PRFA performance for ADP:	40.50		32.89		36.30		56.23		[[172.0, 100.0], [72.0, 49.0]]
- PRFA performance for ADV:	0.49		25.00		0.96		63.43		[[358.0, 3.0], [204.0, 1.0]]
- PRFA performance for NOUN:	7.36		29.63		11.79		70.83		[[1131.0, 76.0], [403.0, 32.0]]
- PRFA performance for PRNOUN:	9.82		22.00		13.58		65.94		[[260.0, 39.0], [101.0, 11.0]]
- PRFA performance for VERB:	17.62		32.72		22.91		59.64		[[796.0, 183.0], [416.0, 89.0]]
- PRFA performance for X:	1.30		26.32		2.47		68.95		[[1742.0, 28.0], [761.0, 10.0]]


In [ ]:
preds = [['sent_index',
 'sent_txt',
 'pos_seq',
 'metaphor_seq',
 'genre',
 'sent_txt_tokenized',
 'sent_bert_tokens',
 'sent_txt_tokenized_bert',
 'BIO_seq',
 'predicitons']]
for i in range(len(predictions)):
  pred = []
  pred.extend(raw_test[i][:2])
  pred.append(str(pred_test[i][-1]))
  pred.extend(raw_test[i][3:])
  pred.append(str(predictions[i].cpu().numpy().tolist()))
  preds.append(pred)

In [ ]:
#write into file
psu_pred = '/content/drive/MyDrive/Metaphor Detection/predict/VUA->PSUCMC_Elmo+FT_BiLSTM_LM.csv'
fh = open(psu_pred, 'w')
writer = csv.writer(fh)
writer.writerows(preds)
fh.close()

In [ ]:
# ADJ evaluation
get_performance_pos(psu_pred, 'ADJ')

ADJ
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		8.33		20.00		11.76		61.54		[[46.0, 8.0], [22.0, 2.0]]
- Fiction:	15.00		18.75		16.67		66.29		[[56.0, 13.0], [17.0, 3.0]]
- Academic:	21.88		30.43		25.45		64.04		[[66.0, 16.0], [25.0, 7.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
15.79		24.49		19.20		64.06		[[168.0, 37.0], [64.0, 12.0]]



array([15.06944444, 23.0615942 , 17.96197267, 63.95522803])

In [ ]:
# ADP evaluation
get_performance_pos(psu_pred, 'ADP')

ADP
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		25.00		28.12		26.47		58.33		[[61.0, 23.0], [27.0, 9.0]]
- Fiction:	41.94		40.62		41.27		70.87		[[77.0, 19.0], [18.0, 13.0]]
- Academic:	50.00		31.76		38.85		41.78		[[34.0, 58.0], [27.0, 27.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
40.50		32.89		36.30		56.23		[[172.0, 100.0], [72.0, 49.0]]



array([38.97849462, 33.50490196, 35.52978346, 56.99343233])

In [ ]:
# ADV evaluation
get_performance_pos(psu_pred, 'ADV')

ADV
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		1.27		50.00		2.47		57.75		[[107.0, 1.0], [78.0, 1.0]]
- Fiction:	0.00		nan		nan		70.00		[[147.0, 0.0], [63.0, 0.0]]
- Academic:	0.00		0.00		nan		61.54		[[104.0, 2.0], [63.0, 0.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
0.49		25.00		0.96		63.43		[[358.0, 3.0], [204.0, 1.0]]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


array([ 0.42194093,         nan,         nan, 63.09749074])

In [ ]:
# All-Pos evaluation
print('All')
get_performance_test(psu_pred)

All
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		4.96		28.06		8.43		64.77		[[1518.0, 100.0], [747.0, 39.0]]
- Fiction:	10.04		30.43		15.09		73.91		[[1729.0, 128.0], [502.0, 56.0]]
- Academic:	12.37		31.41		17.75		59.58		[[1380.0, 238.0], [772.0, 109.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
9.17		30.45		14.09		66.02		[[4627.0, 466.0], [2021.0, 204.0]]



array([ 9.12332618, 29.96814677, 13.75973835, 66.08797731])

In [ ]:
# NOUN evaluation
get_performance_pos(psu_pred, 'NOUN')

NOUN
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		5.66		37.50		9.84		72.03		[[416.0, 15.0], [150.0, 9.0]]
- Fiction:	5.15		17.24		7.94		74.67		[[337.0, 24.0], [92.0, 5.0]]
- Academic:	10.06		32.73		15.38		66.67		[[378.0, 37.0], [161.0, 18.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
7.36		29.63		11.79		70.83		[[1131.0, 76.0], [403.0, 32.0]]



array([ 6.95696082, 29.15621735, 11.0523963 , 71.12435135])

In [ ]:
# Pronoun evaluation
get_performance_pos(psu_pred, 'PRNOUN')

PRNOUN
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		3.12		14.29		5.13		61.86		[[59.0, 6.0], [31.0, 1.0]]
- Fiction:	7.41		30.77		11.94		73.30		[[158.0, 9.0], [50.0, 4.0]]
- Academic:	23.08		20.00		21.43		52.69		[[43.0, 24.0], [20.0, 6.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
9.82		22.00		13.58		65.94		[[260.0, 39.0], [101.0, 11.0]]



array([11.20311016, 21.68498168, 12.83235835, 62.61566986])

In [ ]:
# VERB evaluation
get_performance_pos(psu_pred, 'VERB')

VERB
Genre-specific Performance:
		Precision	Recall		F1		Accuracy	Matrix
- News:		10.81		30.19		15.92		62.53		[[266.0, 37.0], [132.0, 16.0]]
- Fiction:	21.97		34.52		26.85		68.34		[[312.0, 55.0], [103.0, 29.0]]
- Academic:	19.56		32.59		24.44		49.06		[[218.0, 91.0], [181.0, 44.0]]

General Performance
Precision	Recall		F1		Accuracy	Matrix
17.62		32.72		22.91		59.64		[[796.0, 183.0], [416.0, 89.0]]



array([17.44535445, 32.43502712, 22.40556477, 59.97601998])